**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [53]:
!pip install keras-flops

In [54]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
from keras.utils import np_utils

In [55]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [56]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [57]:
# Преобразуйте метки классов в one_hot формат
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [58]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [75]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(8, 3, padding='same'),
    tf.keras.layers.MaxPooling2D(9, strides=9),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [76]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-24 18:56:26.072499: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 18:56:26.072910: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-24 18:56:26.073025: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-24 18:56:26.073450: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 18:56:26.073857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 18:56:26.074185: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

In [77]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 8)         224       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 3, 3, 8)           0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 3, 3, 8)           32        
_________________________________________________________________
activation_18 (Activation)   (None, 3, 3, 8)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 72)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               7300      
_________________________________________________________________
activation_19 (Activation)   (None, 100)              

In [79]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adamax(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=10000, decay_rate=0.3)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [80]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=256
)

Epoch 1/256
391/391 [==============================] - 2s 4ms/step - loss: 3.9398 - accuracy: 0.1082 - val_loss: 3.6931 - val_accuracy: 0.1492
Epoch 2/256
391/391 [==============================] - 1s 4ms/step - loss: 3.4750 - accuracy: 0.1850 - val_loss: 3.4819 - val_accuracy: 0.1819
Epoch 3/256
391/391 [==============================] - 1s 3ms/step - loss: 3.3188 - accuracy: 0.2103 - val_loss: 3.3557 - val_accuracy: 0.2089
Epoch 4/256
391/391 [==============================] - 1s 3ms/step - loss: 3.2412 - accuracy: 0.2267 - val_loss: 3.3446 - val_accuracy: 0.2183
Epoch 5/256
391/391 [==============================] - 1s 3ms/step - loss: 3.1931 - accuracy: 0.2378 - val_loss: 3.3044 - val_accuracy: 0.2230
Epoch 6/256
391/391 [==============================] - 1s 3ms/step - loss: 3.1563 - accuracy: 0.2458 - val_loss: 3.2498 - val_accuracy: 0.2302
Epoch 7/256
391/391 [==============================] - 1s 4ms/step - loss: 3.1264 - accuracy: 0.2512 - val_loss: 3.2516 - val_accuracy: 0.2291

In [65]:
!zip -r file.zip /kaggle/working/

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/20-0.25.hdf5 (deflated 33%)
  adding: kaggle/working/01-0.25.hdf5 (deflated 33%)
  adding: kaggle/working/59-0.26.hdf5 (deflated 33%)
  adding: kaggle/working/122-0.28.hdf5 (deflated 33%)
  adding: kaggle/working/41-0.27.hdf5 (deflated 33%)
  adding: kaggle/working/05-0.23.hdf5 (deflated 33%)
  adding: kaggle/working/11-0.26.hdf5 (deflated 33%)
  adding: kaggle/working/31-0.27.hdf5 (deflated 33%)
  adding: kaggle/working/03-0.21.hdf5 (deflated 33%)
  adding: kaggle/working/57-0.26.hdf5 (deflated 33%)
  adding: kaggle/working/25-0.27.hdf5 (deflated 33%)
  adding: kaggle/working/195-0.28.hdf5 (deflated 33%)
  adding: kaggle/working/37-0.27.hdf5 (deflated 33%)
  adding: kaggle/working/53-0.27.hdf5 (deflated 33%)
  adding: kaggle/working/02-0.21.hdf5 (deflated 33%)
  adding: kaggle/working/20-0.28.hdf5 (deflated 33%)
  adding: kaggle/working/254-0.28.hdf5 (deflated 33%)
  adding: kaggle/working/89-0.28.hdf5 (deflated 33%)
  add